# Neural Style Transfer

```
Credits:
```
Fast Style Transfer is a Neural Style Transfer approach to stylize an image based on an arbitrary image/painting.
<br>
Reference: [Magenta (by Google)](https://github.com/tensorflow/magenta/tree/master/magenta/models/arbitrary_image_stylization) and the publication:
[Exploring the structure of a real-time, arbitrary neural artistic stylization
network](https://arxiv.org/abs/1705.06830)

```
Objective:
```
The idea is to take a rough sketch of a new interior and apply the style corresponding to an already existing project (Eg: An already existing/completed interior design). As a result, the style of the existing interior shall be applied on the rough sketch 

```
Approach:
```
A [pre-trained model available in Tensorflow Hub](https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2) is leveraged in this notebook

## Setup

Import of Tensorflow and all relevant dependencies

In [6]:
import functools
import os
import cv2

from matplotlib import gridspec
import matplotlib.pylab as plt
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub

print("TF Version: ", tf.__version__)
print("TF Hub version: ", hub.__version__)
print("Eager mode enabled: ", tf.executing_eagerly())
print("GPU available: ", tf.config.list_physical_devices('GPU'))

TF Version:  2.10.0
TF Hub version:  0.12.0
Eager mode enabled:  True
GPU available:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


## Constants

In [7]:
MODEL_URL = 'https://tfhub.dev/google/magenta/arbitrary-image-stylization-v1-256/2'
INPUT_DIR = './input/'
OUTPUT_DIR = './output/'
SKETCHES_DIR = INPUT_DIR+'sketches/'
STYLES_DIR = INPUT_DIR+'styles/'

## Reusable Functions

In [8]:
def saveNShowImages(images, titles=('',), saveURL=None, show=False):
    n = len(images)
    image_sizes = [image.shape[1] for image in images]
    w = (image_sizes[0] * 6) // 320
    plt.figure(figsize=(w * n, w))
    gs = gridspec.GridSpec(1, n, width_ratios=image_sizes)
    for i in range(n):
        image = images[i][0]
        plt.subplot(gs[i])
        plt.imshow(image, aspect='equal')
        plt.axis('off')
        plt.title(titles[i] if len(titles) > i else '')
    if saveURL != None:
        plt.savefig(saveURL)
    if show == True:
        plt.show()
    plt.close()

def readImages(cnURL, stURL, cnSize=512, stSize=1024):
    cn1 = tf.io.read_file(cnURL)
    cn2 = tf.io.decode_image(cn1, channels=3, dtype=tf.float32)
    cn3 = cn2[tf.newaxis, ...]
    cn4 = tf.image.resize(cn3, (cnSize, cnSize), preserve_aspect_ratio=True)

    st1 = tf.io.read_file(stURL)
    st2 = tf.io.decode_image(st1, channels=3, dtype=tf.float32)
    st3 = st2[tf.newaxis, ...]
    st4 = tf.image.resize(st3, (stSize, stSize), preserve_aspect_ratio=True)
    st5 = tf.nn.avg_pool(st4, ksize=[3,3], strides=[1,1], padding='SAME')    

    return cn4, st5 

## Pre-trained Hub Model

In [9]:
# Load the pre-trained model from TF Hub
hub_module = hub.load(handle=MODEL_URL)

## Image Stylization

In [10]:
sketch_files = os.listdir(SKETCHES_DIR)
style_files = os.listdir(STYLES_DIR)
for sketch_file in sketch_files:
    for style_file in style_files:
        content_image, style_image = readImages(SKETCHES_DIR+sketch_file, STYLES_DIR+style_file)
        outputs = hub_module(tf.constant(content_image), tf.constant(style_image))
        content_image = tf.image.resize(content_image, (512,512), preserve_aspect_ratio=True)
        style_image = tf.image.resize(style_image, (512,512), preserve_aspect_ratio=True)
        output_image = tf.image.resize(outputs[0], (512,512), preserve_aspect_ratio=True)
        saveURL = OUTPUT_DIR+sketch_file[:-4]+style_file
        saveNShowImages([content_image, style_image, output_image], titles=['Original content image', 'Style image', 'Stylized image'], saveURL=saveURL)
